In [1]:
import os
os.chdir('/home/reffert/DeepAR_InfluenzaForecast')
from PythonFiles.Configuration import Configuration
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime


import os
os.chdir('/home/reffert/DeepAR_InfluenzaForecast')
from PythonFiles.Configuration import Configuration
from PythonFiles.HpTuning import get_data, objectiveDeepAR
from ray.air import session
from ray import tune
data_splits_dict = get_data(truncate=False, with_features=True)[0]

In [ ]:
hp_search_space = {
    "num_cells": tune.grid_search([10, 60, 140]),
    "num_layers": tune.grid_search([6, 12]),
    "context_length":tune.grid_search([1, 2, 4]),
    "cell_type":tune.grid_search(["lstm"]),
    "epochs":tune.grid_search([90, 140, 200]),
    "use_feat_static_real":tune.grid_search([False]),
    "use_feat_dynamic_real":tune.grid_search([True]),
    "use_feat_static_cat":tune.grid_search([False, True]),
    "cardinality":tune.sample_from(lambda spec:[2]*411 if spec.config.use_feat_static_cat else None),
}


train = data_splits_dict["with_features_2001"][0]
test = data_splits_dict["with_features_2001"][1]
configuration = Configuration()

tuner = tune.Tuner(
    tune.with_parameters(objective, train=train, test=test, configuration=configuration),
    tune_config=tune.TuneConfig(
        num_samples = 6,
        metric="mean_WIS",
        mode="min",
        max_concurrent_trials=10,
    ),
    param_space=hp_search_space,
)
results = tuner.fit()

print("Best hyperparameters found were: ", results.get_best_result().config)

results_df = results.get_dataframe()
print(results_df)
results_df.to_csv("Hyperparameter_results_19_05.csv")